<center> <h2> DS 3000 - Fall 2020</h2> </center>
<center> <h3> DS Report </h3> </center>


<center> <h3> Calculating Startup Funding</h3> </center>
<center><h4>Breanna Madison, Kelley Phippen, Thomas Greenwald</h4></center>


<hr style="height:2px; border:none; color:black; background-color:black;">

#### Executive Summary:

Add your summary here (100-150 words)

Provide a brief summary of your project. After reading this executive summary, your readers should have a rough understanding of what you did in this project. You can think of this summary in terms of the four sections of the report and write 1-2 sentences describing each section.

TODO: once project is closer to completion


<hr style="height:2px; border:none; color:black; background-color:black;">

## Outline
1. <a href='#1'>INTRODUCTION</a>
2. <a href='#2'>METHOD</a>
3. <a href='#3'>RESULTS</a>
4. <a href='#4'>DISCUSSION</a>

<a id="1"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 1. INTRODUCTION

### Problem Statement

Finding startups is not difficult, but finding great startups is an extremely challenging task. There's the famously thrown around statistic which states that 9 out of 10 startups will fail. While it’s debated how much validity there is to that statement, one thing is for sure, operating a successful startup is HARD. Startups require lots of money to get off the ground and get moving, but investing money in businesses where most fail is a very risky endeavor. This is usually the onus of angel investors and venture capital (VC) firms, who frequently invest in early stage startups. To make this sort of risky investment requires a lot of effort to source potential winners, and conduct due diligence on them to evaluate the opportunity. We want to help make sourcing startups and evaluating them easier and more intelligent, by creating an assortment of machine learning classifiers and predictors. These classifiers will help group potential startups by their market and product fit, and evaluate them based on common traits of those groups. We seek to be able to predict quantifiable attributes of successful startups, like the potential for capital investments. At the end of the day, we want to learn what makes a winner, or even better, a unicorn. 

### Significance of the Problem

Without startups, we would not have companies pushing the boundaries of technology and innovation. We would have no AirBnB, no Instagram, no Uber, no SpaceX, no Tesla. Startups are where new ideas are born, new industries are created, and new markets that we didn’t even know existed yet. Fostering this sort of innovative ecosystem that drives the world forward is an extremely important, not to mention profitable, endeavor. Oftentimes angels and VC firms can make hundreds of times their investment if the company ends up with a big valuation, creating lots of financial incentive to intelligently source startups. In addition, sourcing is mostly a manual process right now, researching startups on newsletters, going to conferences, reaching out to founders on LinkedIn. If there was a tool that helped automate a lot of this work, and narrow down the amount of companies that needed due diligence, it would be a tremendous help to angels and VCs. Oftentimes there will be thousands of potential startups that angels and VCs need to group and explore, and it takes hours and hours of human labor. With our classifiers and predictors it will cut this time down tremendously, allowing them to focus on what matters, the winners. 

### Questions/Hypothesis

- How can we use various metrics such as market, region, founding month, etc. to predict how much total funding a startup will raise?
 
- Can we use linear regression and the aforementioned metrics to make a prediction on funding?

- Can we determine whether Ridge or Lasso regularization will result in a better linear regression model?

- H0: The 11 feature variables (market, country_code, region, funding_rounds, founded_month, founded_quarter, founded_year, seed, venture, round_a, round_b) will NOT have an effect on predicting the total funding amount a startup raises. 

- H1: The 11 feature variables (market, country_code, region, funding_rounds, founded_month, founded_quarter, founded_year, seed, venture, round_a, round_b) will have an effect on predicting the total funding amount a startup raises. 

<a id="2"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

In [62]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

## 2. METHOD

### 2.1. Data Acquisition

We obtained this data set from an online source called Kaggle. The link can be found here:https://www.kaggle.com/arindam235/startup-investments-crunchbase. This dataset is a condensed version of Crunchbase’s csv dump. It has 49,438 rows and 39 raw columns (38 feature variables and 1 target).

In [41]:
raw_data = pd.read_csv("https://raw.githubusercontent.com/tommygreenwald/DS3000FP/main/investments_VC_original.csv",
                       encoding="latin")
raw_data.head(5)

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2.2. Data Analysis

The IVs are the 11 feature varibles which include market, country_code, region, funding_rounds, founded_month, founded_quarter, founded_year, seed, venture, round_a, and round_b. These IVs try to predict the DV of total funding. We chose these variables as predictors because they intuitively seemed to be the most directly correlated with total funding for a startup based on our prior knowledge of venture capital funding. This is a supervised ML problem because we are trying to use these labeled feature variables to *predict* a continuous target variable. Thus, the learning task is a regression problem. We are going to use linear regression with both LASSO (L1) and Ridge (L2) regularization. We are also going to try kNN regression as well as an SVM regression. We felt that these 3 algorithms, with different types of regularization included, will allow us to get a full picture of the data and account for any patterns in the distribution of the data that may cause one model or regularization to perform better than the others. 

<a id="3"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 3. RESULTS

### 3.1. Data Wrangling
In this section, you should do the following and explain why you are doing what you are doing. For each, you should include your code in a cell, followed by a sample output. For instance, if you are one-hot encoding one of your variables, you should first describe what it is and why you are doing it. You should then include your code in a cell, and the sample output should be available as well.




In [43]:
# Perform simple data cleaning (delete extra columns, deal with NA values, etc.)

# First, we are going to drop the feature columns that we do want to use in our regression task. To do this,
# we will just select the ones that we do want.
raw_data = raw_data[[" market ", "country_code", "region", "funding_rounds", "founded_month", "founded_quarter",
                     "founded_year", "seed", "venture", "round_A", "round_B", " funding_total_usd "]]

# Next, we are going to drop the rows where there are null values for any of the feature variables.
raw_data.dropna(inplace=True)

# Last, we are going to rename some of the columns with more intuitive feature names.
raw_data.rename(columns={" market ": "market", " funding_total_usd ": "funding_total"}, inplace=True)

In [42]:
raw_data.head(5)

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# Perform data wrangling to get your features and target values (e.g., grouping your dataframe by columns, applying functions to format dataframes, etc.)

# First, we are going separate out our features and target into different dataframes.
features = raw_data.drop(columns=["funding_total"])
target = raw_data["funding_total"]

# Next, we are going to apply some functions to format various feature columns.
# Convert floats to ints
int_features = ["funding_rounds", "founded_year", "seed", "venture", "round_A", "round_B"]
for feature in int_features:
    features[feature] = features[feature].apply(np.int64)

# Cut off year from founded month and quarter to get more useful data
year_features = ["founded_month", "founded_quarter"]
for feature in year_features:
    features[feature] = features[feature].apply(lambda x: x[5:])

# Remove unnecessary and weirdly formatted commas from target column
target = target.apply(lambda x: x.replace(",", ""))

,market,country_code,region,funding_rounds,founded_month,founded_quarter,founded_year,seed,venture,round_A,round_B
0,News,USA,New York City,1,06,Q2,2012,1750000,0,0,0
2,Publishing,EST,Tallinn,1,10,Q4,2012,40000,0,0,0
3,Electronics,GBR,London,1,04,Q2,2011,1500000,0,0,0
4,Tourism,USA,Dallas,2,01,Q1,2014,0,0,0,0
5,Software,USA,Ft. Lauderdale,1,10,Q4,2011,0,7000000,0,7000000


In [45]:
features.head(5)

,market,country_code,region,funding_rounds,founded_month,founded_quarter,founded_year,seed,venture,round_A,round_B
0,News,USA,New York City,1,06,Q2,2012,1750000,0,0,0
2,Publishing,EST,Tallinn,1,10,Q4,2012,40000,0,0,0
3,Electronics,GBR,London,1,04,Q2,2011,1500000,0,0,0
4,Tourism,USA,Dallas,2,01,Q1,2014,0,0,0,0
5,Software,USA,Ft. Lauderdale,1,10,Q4,2011,0,7000000,0,7000000


In [44]:
target.head(5)

0     1750000 
2       40000 
3     1500000 
4       60000 
5     7000000 
Name: funding_total, dtype: object

In [57]:
# * Preprocess your variables (e.g., scaling/transforming feature variables to normalize them)

# For the numerical features, we are going to use a standard scaler to standardize the data. This will make the
# mean equivalent to 0 and the standard deviation to 1. For the categorical features, we are going to use a label
# encoder to encode the target labels with values between 0 and number of classes. However, a standard scaler must
# be applied on the training and testing data separately or else the results will not be accurate in context of the
# data. The mean and standard deviation need to be for only the data that is contained after the split. Therefore,
# we will split into testing and training sets after the label encoder is applied and before the standard scaler.

columns_to_encode = ["market", "country_code", "region", "funding_rounds", "founded_month",
                        "founded_quarter", "founded_year"]
columns_to_scale = ["seed", "venture", "round_A", "round_B"]

# Encode all data
le = LabelEncoder()
features[columns_to_encode] = features[columns_to_encode].apply(le.fit_transform)

# Split into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(features, target)

# Scale all data based on testing and training
scaler = StandardScaler().fit(X_train[columns_to_scale])
X_train_scaled = np.concatenate([scaler.transform(X_train[columns_to_scale]), X_train[columns_to_encode]], axis=1)
X_test_scaled = np.concatenate([scaler.transform(X_test[columns_to_scale]), X_test[columns_to_encode]], axis=1)

In [58]:
X_train_scaled[:3]

array([[-3.01609524e-01, -1.02703695e-01, -2.56612482e-01,
        -2.13468452e-01,  8.80000000e+01,  1.05000000e+02,
         7.56000000e+02,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.10000000e+01],
       [-3.01609524e-01,  1.03026207e+00,  1.86214466e+00,
         2.13468787e+00,  7.00000000e+00,  1.05000000e+02,
         7.56000000e+02,  3.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.40000000e+01],
       [-3.01609524e-01, -2.36373417e-01, -7.57564685e-02,
        -2.13468452e-01,  5.60000000e+01,  1.05000000e+02,
         7.76000000e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.40000000e+01]])

In [59]:
X_test_scaled[:3]

array([[-3.01609524e-01, -2.58406887e-01, -2.56612482e-01,
        -2.13468452e-01,  1.92000000e+02,  1.70000000e+01,
         5.94000000e+02,  1.00000000e+00,  5.00000000e+00,
         1.00000000e+00,  9.60000000e+01],
       [-3.01609524e-01, -2.65751378e-01, -2.56612482e-01,
        -2.13468452e-01,  5.00000000e+02,  1.05000000e+02,
         7.02000000e+02,  0.00000000e+00,  9.00000000e+00,
         3.00000000e+00,  9.30000000e+01],
       [ 2.84053597e-01, -1.77617495e-01,  2.85955559e-01,
        -2.13468452e-01,  4.10000000e+02,  3.70000000e+01,
         5.08000000e+02,  1.00000000e+00,  1.00000000e+01,
         3.00000000e+00,  9.60000000e+01]])

In [ ]:
# * Perform feature extraction (dummy variables, new features from existing features, etc.)

In [66]:
# * Use one feature selection technique to select a subset of your original features
# TODO: will be completed at a later date
# models = {"L1 Lin Reg": Lasso(), "L2 Lin Reg": Ridge(),
#           "Random Forest": RandomForestClassifier()}
# for name, model in models.items():
#     print(name)
#     rfe = RFE(model, n_features_to_select=8, step=1)
#     rfe.fit(X_train_scaled, y_train)
#     print(pd.Series(rfe.ranking_, index = features.columns[:-1]).sort_values())

### 3.2. Data Exploration
* Generate appropriate data visualizations for your key variables identified in the previous section
* You should have at least three visualizations (and at least two different visualization types)
* For each visualization provide an explanation regarding the variables involved and an interpretation of the graph.
* If you are using Plotly, insert your visualizations as images as well (upload the graph images to an online source, e.g. github, and embed those into the cells in Jupyter Notebook). This is a requirement.


### 3.3. Model Construction
* Conduct your hypothesis test(s) here.
* For your machine learning question(s), use the Training, Validation, and Testing approach through GridSearch
* Apply machine learning algorithms (apply at least three different algorithms)
* Train your algorithms

### 3.4. Model Evaluation
* Evaluate the performance of your algorithms on appropriate evaluation metrics, using your **validation set**
    * Use at least two different metrics 
* Evaluate your results from multiple ML models and hypothesis tests
    * What was the performance of each algorithm in plain English? Is there any indication of overfitting/underfitting?
    * Was there a significant difference? Use the template from lecture slides when reporting the results of your hypothesis testing.

### 3.5. Model Optimization
* Tune your models using appropriate hyperparameters
* Explain why you are doing this (e.g., to avoid overfitting, etc.)

### 3.6. Model Testing
* Test your tuned algorithms using your **testing set**

<a id="4"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 4. DISCUSSION
* Interpret your findings from 3.4., 3.5, and 3.6
    * Which algorithms did you compare?
    * Which algorithm(s) revealed best performance?
    * Which algorithm(s) should be used for your predictive model?
    * Based on your findings, can we use the features in your dataset to predict the outcome variable you identified using the algorithms you've applied? (It is okay if the answer is no. We're interested in the process, not the performance of the model.)
* For your hypotheses, interpret the results. What does it mean to have significant/non-significant differences with regards to your data?


* End this section with a conclusion paragraph containing some pointers for future work
    * (e.g., get more data/features, perform another analysis, etc.)

<a id="5"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

### CONTRIBUTIONS
* Describe each team member's contributions to the report (who did what in each section)
* Remember this is a team effort!
* Each member of your team will provide peer evaluation of other team members. Your final grade on the project will be based on those peer evaluations. A survey will be shared after the deadline for this deliverable.